In [5]:
import os

GOOGLE_APPLICATION_CREDENTIALS="./user-gcp-sa.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS

In [6]:
#!pip3 install -r requirements.txt

In [7]:
import kfp
client = kfp.Client(host='4709991555f6b0ae-dot-us-central1.notebooks.googleusercontent.com')

# AutoML Serverless ML (Taxi) on KFP

#TODO: set GCS location to experiment run
#TODO: set num passengers out of categorical
- TODO: run on unsampled datset
- TODO: set split column to use fingerprint
- TODO: Difference between table and dataset?
TODO: try using a cleanup task with boolean in testing mode
TODO: does generatinge new stats do anything?
TODO: How to add name to condition
TODO: Existing dataset as '' vs None?
TODO: good documentation on how to trigger code after conditional loop if the conditional doesn't run, we working on more traditional code logic?
TODO: in general, make clear what steps have inputs / outputs when or when they're not executed? get confusing to translate between argo and general python, None, etc
TODO: AUtoML, wanted to make more modular, had to refactor becuase you need to supply target column path, not name, couldnt' do my shortcut where i skipped dataset creation, ideally you only know a priori dataset, and then column names, not full pahts?
(might be more an issue with the op), also seems easies with new tables API?
TODO: AutoML Tables, why is it in core when not supported?
TODO: operand, couldn't compare with None so had to go to emtpy string
AttributeError: 'NoneType' object has no attribute 'id' (happens when client not initiated or stale?)
==None but not is None is supported

Wanted to do some logic on dataset_create_id, but it has to basically be reference as input to another step (not condition) or it will only be an artifact, not a paramter?
also, when i provide a paramater as teh input for another step, the variable name gets replaced in the UI as the paramater id, not the input name?


Ultimatey understanding Argo helped a lot
exit handler variations: https://argoproj.github.io/docs/argo/examples/README.html#exit-handlers

https://github.com/sebinsua/k8s-argo-parallel-aggregate-workflow/blob/master/parallel-aggregate-workflow.yml
https://github.com/argoproj/argo/issues/934

how to programtically delete a model that is training?

need for really basic python params, like LIMIT or adding to a query

bigquery op needs to output the temporary table? so i can suck it into AUtoML tables?

this is ok though?
gs://{{inputs.parameters.gcs_bucket}}/{{inputs.parameters.gcs_temp_directory}}/bq_taxi_output-*.csv
output_gcs_path='gs://{}/{}/bq_taxi_output-*.csv'.format(gcs_bucket, gcs_temp_directory)

why do outputs not put out more in general? i.e. BQ one shows me all the files, as well as the fuzzy match?



In [9]:
# Configuration

PROJECT_ID = 'dhodun1'
COMPUTE_REGION = 'us-central1' # Currently us-central1 is only region
BUCKET = 'dhodun1-central1'

# Raw dataset, not cleaned
QUERY = '''
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
'''

QUERY_CLEAN_100k = '''
CREATE OR REPLACE TABLE `dhodun1.kfp_tmp_dataset.taxi_automl_export`
AS

SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_datetime,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  ST_Distance(ST_GeogPoint(pickup_longitude, pickup_latitude), ST_GeogPoint(dropoff_longitude, dropoff_latitude)) AS euclidean
  , CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek
  , CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday
FROM `nyc-tlc.yellow.trips`
# The full dataset has 1+ Billion rows, let's take only 1 out of 1,000 (or 1 Million total)
#WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 1000000) = 1
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70code
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  LIMIT 100000000'''


## Create Op to change 'passengers' from categorical to numeric
Should be fixed with *1.0 in BQ Query, but related to this BQ CSV export bug: https://b.corp.google.com/issues/143356550

In [61]:
# Test on training model, can I delete it?
from google.cloud import automl_v1beta1
client = automl_v1beta1.TablesClient()

# models = list(client.list_models(project=PROJECT_ID))
# for model in models:
#     print(model.name)

evaluations = client.list_model_evaluations(model_name='projects/978546835329/locations/us-central1/models/TBL8170501692329033728')
for evaluation in evaluations:
    print(evaluation.regression_evaluation_metrics)


#print(type(evals[0].regression_evaluation_metrics))

#client.list_model_evaluations(model=)

#client.get_model_evaluation(model_evaluation_name)

# models = [s for s in client.list_models(project=PROJECT_ID)]
# for model in models:
#     # print(model.deployment_state)
#     print(model.dataset_id)



root_mean_squared_error: 3.5239999294281006
mean_absolute_error: 1.5740498304367065
mean_absolute_percentage_error: 16.357690811157227
r_squared: 0.851056694984436
root_mean_squared_log_error: nan



In [44]:
from typing import NamedTuple
from kfp import dsl
from kfp import components

def get_automl_tables_regression_eval(
    model_name: str,
):
    # Returns AutoML Regressions stats, if true sets as 'metric' as well
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'google-cloud-automl==0.9.0', '--quiet', '--no-warn-script-location'], env={'PIP_DISABLE_PIP_VERSION_CHECK': '1'}, check=True)

    from google.cloud import automl_v1beta1
    client = automl_v1beta1.TablesClient()

    evals = list(client.list_model_evaluations(model_name=model_name))
    for eval in evals:
        if eval.regression_evaluation_metrics is not None:
            return(eval.regression_evaluation_metrics)



In [45]:
get_automl_tables_regression_eval(model_name='projects/978546835329/locations/us-central1/models/TBL8170501692329033728')


root_mean_squared_error: 3.5239999294281006
mean_absolute_error: 1.5740498304367065
mean_absolute_percentage_error: 16.357690811157227
r_squared: 0.851056694984436
root_mean_squared_log_error: nan



In [25]:
from typing import NamedTuple
from kfp import dsl
from kfp import components

def set_automl_tables_column_type(
    dataset_path: str,
    column_display_name: str,
    type_code: str,    
):
    # Updates AutuML Column with new column type, does trigger a new column statistics job? how do we check?
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'google-cloud-automl==0.9.0', '--quiet', '--no-warn-script-location'], env={'PIP_DISABLE_PIP_VERSION_CHECK': '1'}, check=True)
    
    from google.cloud import automl_v1beta1
    client = automl_v1beta1.TablesClient()
    
    dataset = client.get_dataset(dataset_name=dataset_path)
    
    column_specs_response = client.list_column_specs(dataset=dataset)
    column_specs = list(column_specs_response)
    
    for column in column_specs:
        if column.display_name == column_display_name:
            # This kicks off a new statistics job... how to check to see if it's done? Took ~ 1 minute this time
            response = client.update_column_spec(column_spec_name=column.name, dataset=dataset, type_code=type_code)
            print('Updated column: "{}" to type code {}. Generating new statistics now...'.format(column_display_name, type_code))

set_automl_tables_column_type_op = components.func_to_container_op(set_automl_tables_column_type, base_image='python:3.7')

In [26]:
# AutoML Tables components
! git clone https://github.com/kubeflow/pipelines.git


fatal: destination path 'pipelines' already exists and is not an empty directory.


In [9]:
from kfp import components

component_store = components.ComponentStore(url_search_prefixes=['https://raw.githubusercontent.com/kubeflow/pipelines/0.1.36/components/'])

automl_create_dataset_for_tables_op = component_store.load_component('gcp/automl/create_dataset_for_tables')

In [27]:
from kfp import components

component_store = components.ComponentStore(local_search_paths=['./pipelines/components'])

In [28]:
automl_create_dataset_for_tables_op = component_store.load_component('gcp/automl/create_dataset_for_tables')
automl_import_data_from_bigquery_op = component_store.load_component('gcp/automl/import_data_from_bigquery')
automl_import_data_from_gcs_op = component_store.load_component('gcp/automl/import_data_from_gcs')
automl_create_model_for_tables_op = component_store.load_component('gcp/automl/create_model_for_tables')
prediction_service_batch_predict_op = component_store.load_component('gcp/automl/prediction_service_batch_predict')
automl_split_dataset_table_column_names_op = component_store.load_component('gcp/automl/split_dataset_table_column_names')

bigquery_query_op = component_store.load_component('gcp/bigquery/query')
#automl_create_dataset_for_tables_op = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/b3179d86b239a08bf4884b50dbf3a9151da96d66/components/gcp/automl/create_dataset_for_tables/component.yaml')

In [29]:
#automl_create_dataset_for_tables_op(PROJECT_ID, COMPUTE_REGION, 'taxi_data',)

In [30]:
set_automl_tables_column_type(dataset_path=dataset_path, column_display_name='passengers', type_code='FLOAT64')

Updated column: "passengers" to type code FLOAT64. Generating new statistics now...


In [48]:
class ObjectDict(dict):
  def __getattr__(self, name):
    if name in self:
      return self[name]
    else:
      raise AttributeError("No such attribute: " + name)

# Define the pipeline

import kfp
from kfp import dsl

def serverless_automl_taxi(
    gcp_project_id: str,
    gcp_region: str,
    query: str,
    gcs_bucket: str,
    gcs_temp_directory: str = 'ml-taxi/{}'.format(dsl.RUN_ID_PLACEHOLDER),
    temp_dataset_id: str = '',
    temp_table_name: str = '',
    dataset_display_name: str = 'taxi_data',
    dataset_location: str = 'US',
    existing_dataset_path: str = '',
    existing_dataset_id: str = '',
    target_column_name: str = 'fare_amount',
    model_display_name: str = 'taxi_data_model',
    train_budget_milli_node_hours: 'Integer' = 1000,
    bq_input_uri: str ='',
):
    """Pipeline to train on Taxi data"""

    new_dataset = True

    output_gcs_path='gs://{}/{}/bq_taxi_output-*.csv'.format(gcs_bucket, gcs_temp_directory)

    if new_dataset:

        # Create dataset
        create_dataset_task = automl_create_dataset_for_tables_op(
            gcp_project_id=gcp_project_id,
            gcp_region=gcp_region,
            display_name=dataset_display_name,
        )
        '''
        # Query to clean dataset and dump to GCS
        bigquery_export_task = bigquery_query_op(
            query=query,
            project_id=gcp_project_id,
            output_gcs_path=output_gcs_path,
            dataset_location=dataset_location,
        )

        # Query to clean dataset and dump to GCS
        bigquery_export_task = bigquery_query_op(
            query=query,
            project_id=gcp_project_id,
            dataset_location=dataset_location,
            dataset_id=temp_dataset_id,
            table_id=temp_table_name,
        )
        
        # Import data from GCS automl_import_data_from_gcs_op
        import_data_task = automl_import_data_from_gcs_op(
            dataset_path=create_dataset_task.outputs['dataset_path'],
            input_uris=[output_gcs_path],
        ).after(bigquery_export_task)
        

        # Import data from BQ automl_import_data_from_gcs_op
        import_data_task = automl_import_data_from_bigquery_op(
            dataset_path=create_dataset_task.outputs['dataset_path'],
            input_uri=temp_table_name,
        ).after(bigquery_export_task)'''

        # Import data from BQ directly
        import_data_task = automl_import_data_from_bigquery_op(
            dataset_path=create_dataset_task.outputs['dataset_path'],
            input_uri=bq_input_uri,
        )

        # Change 'passengers' column from categorical to numerical
        # Should be fixed with *1.0 in BQ Query, but related to this BQ CSV export bug: https://b.corp.google.com/issues/143356550
        set_passengers_numeric_task = set_automl_tables_column_type_op(
            dataset_path=import_data_task.outputs['dataset_path'],
            column_display_name='passengers',
            type_code='FLOAT64',
        )
    else:
        # Create the objects so that the rest of the pipeline will run
        create_dataset_task = ObjectDict({
            'outputs': {
                'dataset_id': existing_dataset_id
            }
        })
        import_data_task = ObjectDict({
            'outputs': {
                'dataset_path': existing_dataset_path
            }
        })


    # Prepare column schemas
    split_column_specs_task = automl_split_dataset_table_column_names_op(
        dataset_path=import_data_task.outputs['dataset_path'],
        table_index=0,
        target_column_name=target_column_name,        
    )
    if new_dataset:
        split_column_specs_task.after(set_passengers_numeric_task)

    # Train a model
    create_model_task = automl_create_model_for_tables_op(
        gcp_project_id=gcp_project_id,
        gcp_region=gcp_region,
        display_name=model_display_name,
        dataset_id=create_dataset_task.outputs['dataset_id'],
        target_column_path=split_column_specs_task.outputs['target_column_path'],
        # input_feature_column_paths=None, # All non-target columns will be used if None is passed
        # input_feature_column_paths=split_column_specs_task.outputs['feature_column_paths'],
        optimization_objective='MINIMIZE_RMSE',
        train_budget_milli_node_hours=train_budget_milli_node_hours,
    )
    
    from kfp.gcp import use_gcp_secret
    kfp.dsl.get_pipeline_conf().add_op_transformer(use_gcp_secret('user-gcp-sa'))

In [52]:
kfp.compiler.Compiler().compile(serverless_automl_taxi, 'pipeline.zip')

In [53]:
from google.cloud import automl

#PROJECT_ID = 'dhodun1'
#COMPUTE_REGION = 'us-central1' # Currently us-central1 is only region

#location_path = automl.AutoMlClient().location_path(PROJECT_ID, COMPUTE_REGION)
client = kfp.Client(host='4709991555f6b0ae-dot-us-central1.notebooks.googleusercontent.com')
client.create_run_from_pipeline_func(
    serverless_automl_taxi,
    arguments=dict(
        gcp_project_id=PROJECT_ID,
        gcp_region=COMPUTE_REGION,
        query=QUERY,
        gcs_bucket=BUCKET,
        dataset_display_name='taxi_data',
        #existing_dataset_path='projects/978546835329/locations/us-central1/datasets/TBL9144959265608302592',
        #existing_dataset_id='TBL9144959265608302592',
        temp_dataset_id='kfp_tmp_dataset',
        temp_table_name='taxi_export',
        bq_input_uri='bq://dhodun1.kfp_tmp_dataset.taxi_automl_export',
    )
)

RunPipelineResult(run_id=2f99b56d-1d4e-4f74-974b-f3075bef62dd)

In [0]:
!gcloud container clusters list

In [0]:
!gcloud container clusters get-credentials kubeflow-marketplace-1 --zone us-central1-a

In [0]:
dataset_path = 'projects/978546835329/locations/us-central1/datasets/TBL5712864505831096320'
client = automl.AutoMlClient()
list_table_specs_response = client.list_table_specs(dataset_path)
list_table_specs_response
table_specs = [s for s in list_table_specs_response]
print('table_specs=')
print(table_specs)
table_spec_name = table_specs[0].name

list_column_specs_response = client.list_column_specs(table_spec_name)
column_specs = [s for s in list_column_specs_response]
#client.get_column_spec('passengers')
for column in column_specs:
    if column.display_name == 'passengers':
        passenger_column = column.name

print('column_specs=')
#print(column_specs)

column = client.get_column_spec(passenger_column)
print(column)

#client = automl_v1beta1.TablesClient()

client.update_column_spec(column_spec_name=passenger_column, type_code='NUMERIC')

In [0]:
column.name

In [0]:
table_specs[]

In [0]:
kfp.run_pipeline_func_on_cluster(
    serverless_ml_taxi_pipeline,
    arguments = dict(
        gcp_project_id=PROJECT_ID,
        gcp_region=COMPUTE_REGION,
        display_name='taxi_data'
    )
)